This command installs and updates several key Python libraries used for natural language processing and model optimization. The exclamation mark ! allows running shell commands directly from environments like Jupyter Notebook. The pip install command fetches and installs packages: transformers provides pre-trained models and training utilities from Hugging Face, datasets handles efficient dataset loading and preprocessing, accelerate optimizes multi-GPU and distributed training, ray[tune] enables scalable hyperparameter tuning, and optuna offers an alternative, efficient framework for automated hyperparameter optimization. The -U flag ensures all packages are upgraded to their latest compatible versions.

In [1]:
!pip install transformers datasets accelerate ray[tune] optuna -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take

**Function Description**

This script conducts random hyperparameter search and fine-tuning for a binary RoBERTa-based text classifier on a mental-health Twitter dataset. It prepares and tokenizes the data, defines a model initialization routine and evaluation metrics, uses Optuna via the Hugging Face Trainer to sample hyperparameters from continuous and categorical distributions, and then retrains a final model using the best hyperparameters discovered by the random search.

**Syntax Explanation**

The code uses Hugging Face transformers and datasets together with scikit-learn metrics and Optuna for hyperparameter sampling. The AutoTokenizer and AutoModelForSequenceClassification classes are used to load model-specific tokenization and weights from the chosen pretrained checkpoint. The dataset is converted from pandas to a Hugging Face Dataset and tokenized with truncation, padding, and max_length=128. The Trainer is configured with model_init so every trial starts from a fresh pretrained model, compute_metrics computes accuracy, F1, precision, and recall using numpy and scikit-learn, and trainer.hyperparameter_search runs a specified number of random trials where Optuna suggests values using functions such as suggest_loguniform, suggest_categorical, suggest_float, and suggest_int.

**Inputs**

The primary input is the CSV file Mental-Health-Twitter.csv, which must contain post_text and label columns; the script renames the label column to labels for Trainer compatibility. The data is cleaned to remove missing or empty post_text entries, then split stratified by label into training and evaluation sets, with the training set sampled to 10,000 rows and the evaluation set sampled to 2,000 rows. The pretrained model identifier margotwagner/roberta-psychotherapy-eval is supplied to initialize tokenizer and model weights.

**Outputs**

The script prints device information, dataset sizes, label distributions, and the number of random trials to run. During the search, per-trial evaluation results are logged through Trainer and Optuna. After the search completes, the best trial and its hyperparameters are printed, a final model is trained with those hyperparameters, and the final evaluation metrics are shown. The final model checkpoint is saved under ./final_model_mental_health_random and the evaluation metrics include loss, accuracy, F1-score, precision, and recall.

**Code Flow**

The script begins by importing libraries, setting a reproducible seed, and selecting the compute device. It loads the CSV dataset, removes missing or empty post_text, renames the label column to labels, and performs a stratified train validation split, followed by sampling to 10,000 training and 2,000 evaluation examples. The DataFrames are converted to Hugging Face Dataset objects and tokenized with truncation, padding, and a max length of 128 tokens, then formatted as PyTorch tensors. A model_init function is defined to instantiate a fresh AutoModelForSequenceClassification for each trial, and compute_metrics is defined to compute accuracy, F1, precision, and recall using predicted class indices. The random_hp_space function defines the random search space using Optuna suggestion methods, sampling learning rate from a log uniform range, selecting batch sizes from a categorical set, sampling weight decay from a float range, and sampling number of epochs as an integer. TrainingArguments sets common training behaviors, and the Trainer is created with model_init, datasets, tokenizer, and metric function. trainer.hyperparameter_search runs NUM_RANDOM_TRIALS random trials with the Optuna backend and maximizes eval_f1 via the provided objective function. If a best trial is found, the script reconstitutes training arguments with the chosen hyperparameters, reinitializes a Trainer, trains the final model, evaluates it, and saves the final checkpoint.

**Comments and Observations**

The use of Optuna for random search allows efficient exploration of continuous and discrete hyperparameter spaces, and suggest_loguniform for learning rate is appropriate because learning rates often span orders of magnitude. Setting model_init ensures trial independence, and optimizing for F1 is suitable for binary labels that may be imbalanced. The chosen sample sizes and max_length=128 are well matched to short social posts, but confirm token length distribution to avoid unnecessary truncation. The code assumes the pandas index column has the name __index_level_0__ when removing columns; verify this to prevent errors. Consider adding Optuna pruning callbacks or using early stopping to abort low-performing trials early and save compute. If GPU memory becomes limiting, reduce per-device batch size, enable gradient accumulation to simulate larger batches, or lower max_length. Finally, validate that labels contains only 0 and 1 and inspect class balance so metrics are interpretable and the final model is reliable.

In [2]:
# 1. SETUP AND INSTALLATION
# Run this command first in your Colab notebook:
# !pip install transformers datasets accelerate ray[tune] optuna pandas -U

import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna # Import optuna to use its suggestion methods for random search

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")

# --- 2. DATA PREPARATION (Using your Mental-Health-Twitter.csv) ---

# Load your dataset
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

# Filter out rows where 'post_text' is NaN or empty
df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']

# Rename 'label' to 'labels' for Hugging Face Trainer compatibility
df = df.rename(columns={"label": "labels"})

# Split data into training and validation sets
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=10000, random_state=42) # Limit to 10k training samples
eval_df = eval_df.sample(n=2000, random_state=42)   # Limit to 2k evaluation samples

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

# Initialize Tokenizer for your specific model
MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

# Set format to PyTorch tensors
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])


# --- 3. MODEL, METRICS, AND HYPERPARAMETER DEFINITION ---

# Function to initialize a fresh model for each grid search run
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# --- HYPERPARAMETER SEARCH SPACE FOR RANDOM SEARCH ---
def random_hp_space(trial):
    """
    This function defines the hyperparameter search space for Random Search.
    Optuna's suggestion methods (like suggest_loguniform, suggest_float) are
    inherently suitable for random sampling from a distribution.
    """
    # 1. Learning Rate (log-uniform distribution is common for learning rates)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5) # Broader range than grid search

    # 2. Batch Size (categorical, as before)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [16, 32])

    # 3. Weight Decay (uniform float distribution)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01) # Broader range

    # 4. Number of Training Epochs (integer uniform distribution)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4) # Allow 2, 3, or 4 epochs

    # --- EXPANSION SUPPORT: Add more parameters here if needed ---
    # Example: gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])

    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
    }


# --- 4. TRAINING ARGUMENTS (Fixed for all runs) ---
training_args = TrainingArguments(
    output_dir="./random_search_results_mental_health",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none",
    num_train_epochs=3, # Placeholder, will be suggested by random_hp_space
    warmup_steps=100,
    logging_dir="./logs_random",
    logging_steps=500,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
)

# Initialize the Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# --- Define the objective function for Optuna ---
def optuna_hp_objective(metrics):
    """
    Optuna objective function that returns the F1 score for maximization.
    `metrics` is the dictionary returned by trainer.evaluate().
    """
    return metrics["eval_f1"]


# --- 5. EXECUTION OF RANDOM SEARCH ---
print("\n--- Starting Random Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

# For random search, we don't calculate total_trials from a grid.
# Instead, we define 'n_trials' (the number of random samples to try).
# Let's aim for 20-30 trials to have a good chance of finding a strong configuration.
# You can adjust this based on your remaining time and observed trial duration.
NUM_RANDOM_TRIALS = 25 # Increased to 25 to ensure more than 10 experiments

print(f"Number of random trials to run: {NUM_RANDOM_TRIALS}")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=random_hp_space, # <--- Use the new random_hp_space function
    direction="maximize",
    n_trials=NUM_RANDOM_TRIALS, # <--- Specify the number of random trials
    compute_objective=optuna_hp_objective,
)

print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

# Extract and print the best configuration
if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health_random", # New output directory
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        warmup_steps=100,
        logging_dir="./final_logs_random", # New logging directory
        logging_steps=500,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters from Random Search...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health_random'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")

Using GPU: Tesla T4
Dataset loaded successfully.
Using 10000 training samples and 2000 evaluation samples.
Train label distribution:
labels
1    0.5004
0    0.4996
Name: proportion, dtype: float64
Eval label distribution:
labels
1    0.5
0    0.5
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[I 2025-11-08 07:51:43,964] Trial 23 pruned. 
/tmp/ipython-input-3164346945.py:104: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5) # Broader range than grid search


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.492200,0.272130,0.881500,0.888260,0.840321,0.942000
2,0.246600,0.319797,0.902500,0.904924,0.882969,0.928000
3,0.174600,0.334233,0.911000,0.912745,0.895192,0.931000


[I 2025-11-08 07:57:41,152] Trial 24 pruned. 



--- Random Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='13', objective=0.9173760640961443, hyperparameters={'learning_rate': 3.727957465978101e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.05, 'num_train_epochs': 4}, run_summary=None)

Best Hyperparameters:
  learning_rate: 3.727957465978101e-05
  per_device_train_batch_size: 16
  weight_decay: 0.05
  num_train_epochs: 4


AttributeError: 'BestRun' object has no attribute 'metrics'